# Coopetition of Danish banks

This notebook contains a simple model to verify the data.

## Step 1. Load KPI spreadsheet from Finanstilsynet

In [19]:
import pandas as pd

# PI_D is a hidden sheet that contains the data for pivot tables PI_A & PI_B
df = pd.read_excel(io="data/KPI_Penge_fonds_Real_202412.xlsx", sheet_name="PI_D")
df.rename(columns={"vaerdi": "value",
                   "År (Year)": "year",
                   "Vælg nøgletal (Choose KPI)": "kpi",
                   "Virksomhedsnavn (Company Name)": "bank"
                   }, inplace=True)
kpi_map = {
    "Solvensprocent (Solvency ratio)": "solvency_ratio",
    "Kernekapitalprocent (Tier 1 capital ratio)": "tier1_cap",
    "Egenkapitalforrentning før skat (Return on equity before tax)": "roe_pre_tax",
    "Egenkapitalforrentning efter skat (Return on equity after tax)": "roe_post_tax",
    "Indtjening pr. omkostningskrone (Income/cost ratio)": "cost_to_income",
    "Renterisiko (Interest rate risk)": "interest_rate_risk",
    "Valutaposition (Foreign exchange position)": "fx_position",
    "Valutarisiko (Foreign exchange risk)": "fx_risk",
    "Udlån plus nedskrivninger herpå i forhold til indlån (Loans and impairment losses over deposits)": "loans_over_deposits",
    "Overdækning i forhold til lovkrav om likviditet (Excess coverage as a percentage of the liquidity requirement)": "liq_coverage",
    "Summen af store eksponeringer (Sum of large exposures)": "large_exposures",
    "Årets nedskrivningsprocent (Annual impairment ratio)": "impairment_ratio",
    "Årets udlånsvækst (Growth in loans)": "loan_growth",
    "Udlån i forhold til egenkapital (Gearing)": "gearing",
    "Årets resultat pr. aktie (stykstørrelse 100 kr.) (Annual earnings per share (nominal value DKK 100) before tax)": "eps_pre_tax",
    "Indre værdi pr. aktie (stykstørrelse 100 kr.)(Book value over net asset value)": "book_value_per_share",
    "Børskurs/resultat pr. aktie (stykstørrelse 100 kr.) (Price over net asset value (nominal value DKK 100))": "price_over_nav",
    "Børskurs/indre værdi pr. aktie (stykstørrelse 100 kr.) (Price over book value (nominal value DKK100))": "price_over_book"
}
df["kpi"] = df["kpi"].map(kpi_map).fillna(df["kpi"])


## Step 2. Add datacentral information

In [20]:
df_platforms = pd.read_excel("data/BankByDC.xlsx")
df = df.merge(df_platforms, on="bank", how="left")
df = df.dropna(subset=["dc"])

## Step 3. Simple model
cost-to-income on various DCs in 2024

In [21]:
import statsmodels.api as sm

# let's focus only on 2024
df_2024 = df[df["year"] == 2024]

# from long to wide format
df_2024 = df_2024.pivot_table(index=["bank", "dc"], columns="kpi", values="value").reset_index()

# banks can be on one of three datacentrals: BEC, NBS or Bankdata
# or can be a datacentral for themselves
# let's drop those special banks
df_2024["dc"] = pd.Categorical(
    df_2024["dc"],
    categories=["OWN", "BEC", "NBS", "Bankdata"],
    ordered=True
)
df_2024 = df_2024[df_2024.dc != "OWN"]
# drop_first = True removes first column (OWN)
df_2024 = pd.get_dummies(df_2024, columns=["dc"], drop_first=True, dtype=int)

y = df_2024["cost_to_income"]
X = df_2024[["tier1_cap", "loan_growth", "gearing", "loans_over_deposits", "dc_NBS", "dc_Bankdata"]]
X = sm.add_constant(X)

# Used robust standard errors (HC3) to account for heteroskedasticity
model = sm.OLS(y, X).fit(cov_type='HC3')
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:         cost_to_income   R-squared:                       0.543
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     1.339
Date:                Sun, 07 Dec 2025   Prob (F-statistic):              0.265
Time:                        21:02:56   Log-Likelihood:                -15.198
No. Observations:                  44   AIC:                             44.40
Df Residuals:                      37   BIC:                             56.89
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   1.1040    

The model is far from being perfect. Small sample, does not explain anything, F-statistic nonsignificant.

## Step 4. Panel Data Analysis
Using the full dataset (all years) to control for time fixed effects and increase sample size.


In [27]:
df_panel = df.pivot_table(index=["bank", "year", "dc"], columns="kpi", values="value").reset_index()

# same as before
df_panel["dc"] = pd.Categorical(
    df_panel["dc"],
    categories=["OWN", "BEC", "NBS", "Bankdata"],
    ordered=True
)
df_panel = df_panel[df_panel.dc != "OWN"]
df_panel = pd.get_dummies(df_panel, columns=["dc"], drop_first=True, dtype=int)

# create year dummies (time fixed effects)
year_dummies = pd.get_dummies(df_panel["year"], prefix="year", drop_first=True, dtype=int)

y_panel = df_panel["cost_to_income"]
X_panel = df_panel[["tier1_cap", "loan_growth", "gearing", "loans_over_deposits", "dc_NBS", "dc_Bankdata"]]

# add year dummies
X_panel = pd.concat([X_panel, year_dummies], axis=1)
X_panel = sm.add_constant(X_panel)

# some last minute data cleaning (TODO: move it earlier)
import numpy as np

mask_valid = (
    ~X_panel.isna().any(axis=1) & 
    ~np.isinf(X_panel).any(axis=1) & 
    ~y_panel.isna() & 
    ~np.isinf(y_panel)
)

X_clean = X_panel.loc[mask_valid]
y_clean = y_panel.loc[mask_valid]
groups_clean = df_panel.loc[mask_valid, 'bank']

print(f"Original samples: {len(X_panel)}, Samples after cleaning: {len(X_clean)}")

model_panel = sm.OLS(y_clean, X_clean).fit(cov_type='cluster', cov_kwds={'groups': groups_clean})
print("Panel Data Model Results:")
print(model_panel.summary())

Original samples: 1067, Samples after cleaning: 1063
Panel Data Model Results:
                            OLS Regression Results                            
Dep. Variable:         cost_to_income   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     26.22
Date:                Sun, 07 Dec 2025   Prob (F-statistic):           8.16e-22
Time:                        21:41:15   Log-Likelihood:                -683.91
No. Observations:                1063   AIC:                             1430.
Df Residuals:                    1032   BIC:                             1584.
Df Model:                          30                                         
Covariance Type:              cluster                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------

A little bit better than simple OLS. Massive multicollinearity and autocorrelation (DW = 0.855). Kurtosis 19, JB higher than the empire state :(

TODO: macro variables